In [1]:
!python --version

Python 3.9.16


In [68]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/gdrive')

embedings_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/assets/task-3/test-embeddings.tsv'
test_df_url= 'gdrive/My Drive/Colab Notebooks/nlp-2023/test.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# считать вектора и запустить классификатор SVM

In [60]:
test_df = pd.read_csv(test_df_url)
y_dataset = list(test_df['class'])

y_train = y_dataset[:7500]
y_test = y_dataset[7500:]

In [18]:
# 1. С использованием полученных в результате выполнения четвертой лабораторной работы векторных представлений документов осуществить эксперименты с моделью SVM для многоклассовой классификации.
columns = [f"emb_{x}" for x in range(0,100)]
embedings = pd.read_table(embedings_url, names=columns)

In [63]:
x_dataset = []
for value in embedings.values:
  x_dataset.append(value.flatten().tolist())

x_train = x_dataset[:7500]
x_test = x_dataset[7500:]

In [116]:
# metrics calculating
def confusion_matrix(actuals, predicted):
  df = pd.crosstab(actuals, predicted, rownames=['Actual'], colnames=['Predicted'])
  for index, row in df.iterrows():
    if index not in df.columns:
      df[index] = 0
  return df

def accuracy(conf_matrix_df):
  sum_all = conf_matrix_df.values.sum()
  pos = 0
  for index, row in conf_matrix_df.iterrows():
    try:
      pos += conf_matrix_df[index].loc[index]
    except:
      pass
  return round(pos/sum_all, 5)

def precision(conf_matrix_df):
    return round((np.diag(conf_matrix_df) / np.sum(conf_matrix_df, axis=0)).mean(axis=0), 5)

def recall(conf_matrix_df):
    return round((np.diag(conf_matrix_df) / np.sum(conf_matrix_df, axis=1)).mean(axis=0), 5)

def f1(precision, recall):
    return round((2 * precision * recall / (precision + recall)), 5)

In [112]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from time import time

import warnings
warnings.filterwarnings("ignore")

scaler = StandardScaler() # для нормализации фич
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

def model_experiment(model_name, model):
  time_start = time()
  model.fit(X_train, y_train)
  time_end = time()

  actual_classes = y_test
  predicted_classes = []
  for test in X_test:
    predicted_classes.append(model.predict([test])[0])
  
  conf_matrix = confusion_matrix(actual_classes, predicted_classes)
  model_accuracy = accuracy(conf_matrix)
  model_precision = precision(conf_matrix)
  model_recall = recall(conf_matrix)
  model_f1 = f1(model_precision, model_recall)
  print(f"\t{model_name} metrix:")
  print(f"\t\t train time:{time_end - time_start}")
  print(f"\t\t conf_matrix:\n{conf_matrix.to_string()}")
  print(f"\t\t accuracy:{model_accuracy}") # Accuracy represents the number of correctly classified data instances over the total number of data instances.
  print(f"\t\t precision:{model_precision}") # Precision should ideally be 1. Точность TP/(TP+FP)
  print(f"\t\t recall:{model_recall}") # Recall should ideally be 1 (high) for a good classifier. Отклик TP/(TP+FN)
  print(f"\t\t f1:{model_f1}") # F1 score is the harmonic mean of precision and recall and is a better measure than accuracy
  print(f"--------------------------------------------------------------------")

In [115]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from time import time

scaler = StandardScaler() # для нормализации фич
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

for epochs_count in range(100, 1001, 100):
  print(f"Epochs count - {epochs_count}")
  
  models = {
    'linear': make_pipeline(StandardScaler(), svm.SVC(kernel='linear', degree=2, max_iter=epochs_count)),
    'poly2': make_pipeline(StandardScaler(), svm.SVC(kernel='poly', degree=2, max_iter=epochs_count)),
    'poly3': make_pipeline(StandardScaler(), svm.SVC(kernel='poly', degree=3, max_iter=epochs_count)),
    'poly4': make_pipeline(StandardScaler(), svm.SVC(kernel='poly', degree=4, max_iter=epochs_count)),
    'rbf': make_pipeline(StandardScaler(), svm.SVC(kernel='rbf', max_iter=epochs_count)),
    'sigmoid': make_pipeline(StandardScaler(), svm.SVC(kernel='sigmoid', max_iter=epochs_count)),
  }
  
  for model_name, model in models.items():
    model_experiment(model_name, model)

Epochs count - 100
	linear metrix:
		 train time:0.2783324718475342
		 conf_matrix:
Predicted   1   2   3   4
Actual                   
1          12   8   3   4
2           5  11   4   5
3           3   6  11   7
4           0   3   5  13
		 accuracy:0.47
		 precision:0.47985
		 recall:0.47772
		 f1:0.47878
--------------------------------------------------------------------
	poly2 metrix:
		 train time:0.40836572647094727
		 conf_matrix:
Predicted  1  2   4  3
Actual                
1          1  0  26  0
2          1  5  19  0
3          2  0  25  0
4          0  0  21  0
		 accuracy:0.27
		 precision:0.50824
		 recall:0.29074
		 f1:0.36989
--------------------------------------------------------------------
	poly3 metrix:
		 train time:0.5530836582183838
		 conf_matrix:
Predicted  1  2  3   4
Actual                
1          5  0  3  19
2          0  7  0  18
3          0  0  0  27
4          0  0  0  21
		 accuracy:0.33
		 precision:0.56176
		 recall:0.3663
		 f1:0.44345
--------

In [ ]:
# 2. После выполнения экспериментов на основании полученных значений метрик сформулировать вывод о том, какое количество итераций является оптимальным, 
  # а также какая модель (или kernel function) показала лучшие результаты (в том случае, если в предыдущем пункте выполнялись эксперименты с несколькими моделями или kernel functions);

# По полученным метрикам rbf в качестве kernel function показала лучшие результаты. 
  # Моя гипотеза такова, что оптимальным количеством эпох будет 800, так как метрики начинают выходить на свои плато

In [ ]:
# 3. В соответствии с одним из следующих преобразований над векторными представлениями документов осуществить еще одну серию экспериментов 
  # над выбранной на предыдущем шаге моделью с использованием найденного оптимального значения для количества итераций (варианты приведены в порядке увеличения сложности реализации):

  